# Analysis of varying trimming thresholds of microbial communities of known composition

Script for the analysis and figure generation o

In [1]:
#Import libraries in python3 kernel
import pandas as pd
import seaborn as sns
import glob
import os
import boto
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
import numpy as np
import skbio
#import fastcluster #this package makes skbio run faster clustermaps but can be tricky with missing values from pairwise comparisons
from functools import reduce
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.SeqUtils import GC
from collections import defaultdict
from collections import Counter
import statistics
import itertools as it
from scipy import stats
from matplotlib.ticker import FormatStrFormatter
from qiime2 import Artifact
import tempfile
import zipfile
import yaml
%matplotlib inline

## Load functions

__consolidate_tables__ creates a dataframe of all the merged feature tables and parameters

In [ ]:
# Special thanks to Alex Manuele https://github.com/alexmanuele
def consolidate_tables(community):
    if community == "16S":
        comm = '02-PROKs'
    if community == "18S":
        comm = '02-EUKs'


    table_list = glob.glob('{0}*/03-DADA2d/table.qza'.format(comm+'/all_trims/'))
    print "Found all tables"

    dataframes = []  
    for table_path in table_list:
        with tempfile.TemporaryDirectory() as tempdir:
            #load table, dump contents to tempdir
            table = Artifact.load(table_path)
            #Make sure the tables are all FeatureFrequency type
            assert str(table.type) == 'FeatureTable[Frequency]', "{0}: Expected FeatureTable[Frequency], got {1}".format(table_path, table.type)
            Artifact.extract(table_path, tempdir)
            #get the provenance form the tempdir and format it for DF
            prov = '{0}/{1}/provenance/'.format(tempdir, table.uuid)
            action = yaml.load(open("{0}action/action.yaml".format(prov), 'r'), Loader=yaml.BaseLoader)
            paramlist = action['action']['parameters']
            paramlist.append({'table_uuid': "{}".format(table.uuid)})
            paramdict = {}
            for record in paramlist:
                paramdict.update(record)

            # Get the data into a dataframe
              #Biom data
            df = table.view(pd.DataFrame).unstack().reset_index()
            df.columns = ['feature_id', 'sample_name', 'feature_frequency']
            df['table_uuid'] = ["{}".format(table.uuid)] * df.shape[0]
              #param data
            pdf = pd.DataFrame.from_records([paramdict])
              #merge params into main df
            df = df.merge(pdf, on='table_uuid')


            #I like having these columns as the last three. Makes it more readable
            cols = df.columns.tolist()
            reorder = ['sample_name', 'feature_id', 'feature_frequency']
            for val in reorder:
                cols.append(cols.pop(cols.index(val)))
            df = df[cols]
            df['table_path'] = [table_path] * df.shape[0]
            dataframes.append(df)

    #Stick all the dataframes together
    outputfile="merged_all_tables.tsv"
    df = pd.concat(dataframes)
    df.to_csv(comm+outputfile, sep='\t', index=False)
    print("Success.")
    return df

__merge_metadata__ adds the metadata to the merged feature tables

In [ ]:
def merge_metadata():
    #df = pd.read_csv('02-PROKs/'+'/merged_all_tables.tsv', sep='\t')

    tables = df[['sample_name', 'feature_id', 'feature_frequency']].copy()
    tables.rename(columns={'sample_name':'file'}, inplace=True)
    manifest = pd.read_csv('MANIFEST.tsv', sep='\t')
    manifest['file'] = [s.split('SPOT_USC_2/')[1] for s in manifest['absolute-filepath']]
    manifest['file'] = [s.split('.R')[0] for s in manifest['file']]
    manifest = manifest.drop(columns = ['absolute-filepath', 'direction'])
    manifest.drop_duplicates()
    merged = pd.merge(tables,manifest, on='file')
    merged = merged.drop(columns = ['file'])
    merged = merged.drop_duplicates() 
    print('Set up manifest ...')
    
    metadata = pd.read_csv('METADATA.tsv', sep='\t')
    merged = pd.merge(merged,metadata, on='sample-id')
    merged = merged.replace({'V2': '16S'}, regex=True)
    print('Set up metadata ...')
    
    merged.to_csv(comm+'filtering_asvs.tsv', sep = '\t')
    print('Saved filtering_asvs.tsv')
    
    return merged

__pick_metadata__ extracts the features according to the given metadata parameters

In [ ]:
def pick_metadata(composition, runnumber, R='all', F='all')
#make df of features/composition+run+comm

    composition = composition
    runnumber = runnumber
    R = R
    F = F

    files = glob.glob('{0}*.tsv'.format(comm+'/all_taxonomies_'+community+'/'))
    taxos = []
#    if not os.path.exists(path+composition):
#        os.mkdir(path+composition)
    for filename in files:
        tax = pd.read_csv(filename, sep='\t')
        tax['table_id'] = str(filename.split('/')[-1])
        tax["table_id"] = tax["table_id"].str.replace(".tsv", "")
        tax['Forward_trim'], tax['Reverse_trim'] = tax['table_id'].str.split('R', 1).str
        tax['Forward_trim'] = tax['Forward_trim'].map(lambda x: x.lstrip('F'))
        tax["Forward_trim"] = pd.to_numeric(tax["Forward_trim"])
        tax["Reverse_trim"] = pd.to_numeric(tax["Reverse_trim"])
        taxos.append(tax)
    print('Appended all taxonomies to taxos')
    taxos = pd.concat(taxos)
    taxos = taxos.rename(columns={"Feature ID": "feature_id"}, errors="raise")
    separated = taxos.merge(merged, how='left', on='feature_id')
    separated = separated.drop_duplicates()
    separated = separated[separated["community"] == community]
    separated = separated[separated["composition"] == composition]
    separated['run-number']= separated['run-number'].astype(str)
    separated = separated[separated["run-number"] == runnumber]
    separated['sum'] = separated.groupby(['table_id','sample-id'])['feature_frequency'].transform('sum')
    separated['ratio'] = separated['feature_frequency']/(separated['sum'])
    separated_taxonomies = separated.copy()
    #make a dictionary with keys for id-ing the taxon belonging to this sub-community
    separated_dic = pd.Series(separated.Taxon.values,separated.feature_id.values).to_dict()
    
    return separated_taxonomies, separated_dic

__rename_taxonomies__ extracts zipped classification files from qiime2, renames them by the trimming length used, and moves them to a new folder

In [ ]:
def rename_taxonomies()
#generate folder of split taxonomies by runnumber and composition
    # Directory
    directory = composition+runnumber
    # Parent Directory path
    parent_dir = 'input_data/'+community+'/all_taxonomies_'+community
    # Path
    path = os.path.join(parent_dir, directory)
    # Create the directory
    # 'GeeksForGeeks' in
    # '/home / User / Documents'
    os.mkdir(path)
    for filename in files:
        taxonomy = pd.read_csv(filename, sep='\t')
        taxonomy = taxonomy.rename(columns={"Feature ID": "feature_id"}, errors="raise")
        newz = taxonomy.merge(merged, how='left', on='feature_id')
        #new = newz.drop(['sample-id'], axis=1)
        new = newz.drop_duplicates()
        new = new[new["community"] == community]
        new = new[new["composition"] == composition]
        new['run-number']= new['run-number'].astype(str)
        new = new[new["run-number"] == runnumber]
        new = new[new.feature_frequency != 0]
        new = new.rename(columns={"feature_id":"Feature ID"}, errors="raise")
        new = new[['Feature ID', 'Taxon', 'Confidence']].copy()
        new = new.drop_duplicates()
        new.to_csv(filename.split('all_taxonomies_'+community)[0]+'all_taxonomies_'+community+'/'+composition+runnumber+'/'+runnumber+filename.split('all_taxonomies_'+community+'/')[1], sep = '\t') 
    
    return new

__make_fasta__ extracts sequences from zipped qiime2 files, and makes new fasta file for the features from the given metadata parameters

In [ ]:
def make_fasta()
# Get list of all .qza
    repseqs = glob.glob('{0}*/*/representative_sequences.qza'.format('/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/all_trims/'), recursive=True)
    for repseq in repseqs:
        with ZipFile(repseq, 'r') as zipObj:
            # Get a list of all archived file names from the zip
            listOfFileNames = zipObj.namelist()
            # Iterate over the file names
            for fileName in listOfFileNames:
                # Check filename endswith fasta
                if fileName.endswith('.fasta'):
                    # Extract a single file from zip
                    zipObj.extract(fileName, 'temp_fasta')
                    
    fastaoutfilename = 'merged'+community+composition+runnumber+R+F+'.fasta'
    
    with open(fastaoutfilename, 'wb') as outfile:
        for filename in glob.glob('temp_fasta/*/*/*.fasta'):
            if filename == fastaoutfilename:
                # don't want to copy the output into the output
                continue
            with open(filename, 'rb') as readfile:
                shutil.copyfileobj(readfile, outfile)
    shutil.rmtree('temp_fasta', ignore_errors=False, onerror=None)
    

    if R!='all':
        rallfs = separated[separated.Reverse_trim == R]
        separated_dic = pd.Series(rallfs.Taxon.values,rallfs.feature_id.values).to_dict()
    else:
        separated_dic = pd.Series(separated.Taxon.values, separated.feature_id.values).to_dict()
    if F!='all':
        fallrs = separated[separated.Forward_trim == F]
        separated_dic = pd.Series(fallrs.Taxon.values,fallrs.feature_id.values).to_dict()
    else:
        separated_dic = pd.Series(separated.Taxon.values, separated.feature_id.values).to_dict()

    fa = SeqIO.parse('input_data/'+community+'/'+fastaoutfilename,
                 "fasta")
    seqs_i_want = [] #we'll put the good sequences here
    for record in fa: #a SeqRecord has the accession as record.id, usually.
        if record.id in separated_dic.keys(): #This is how you check if the accession is in the values of the dict
            seqs_i_want.append(record)
    #Now we can write the list of records to a fasta file. This will take care of the formatting etc
    with open('input_data/'+community+composition+runnumber+R+F+'.fasta', "w") as f:
        SeqIO.write(seqs_i_want, f, "fasta")
    
    return print('Saved selected sequences as' 'input_data/'+community+'/'+runnumber+composition+'R'+R+'F'+F+'.fasta')

__make_tbd_hm__ makes a heatmap showing the taxonomic beta diversity of each trim length combination against the expected community. TBD is a dissimilarity index based on taxonomic trees between two samples where 1 is completely different, and 0 is the completely the same.

In [ ]:
def make_tbd_hm(level=7, truthfilename):
    if comm == '02-PROKs':
        expected_file = 'expected_16s_staggered'
    if comm='02-EUKs':
        expected_file = 'expected_18s_staggered'
    
    bacaros_dm = pd.read_csv(comm+'/Bacaros/results-'+level)
    bacaros_dm = bacaros_dm.set_index('Unnamed: 0')
    bacaros_dm = 1  - bacaros_dm
    #bacaros_dm is a distance matrix of table X table
    my_pcoa = skbio.stats.ordination.pcoa(bacaros_dm.values)
    plt.scatter(my_pcoa.samples['PC1'],  my_pcoa.samples['PC2'])
    against_exp = bacaros_dm[[truthfilename]].copy()
    against_exp = against_exp.reset_index().rename(columns={against_exp.index.name:'sample_name'})
    against_exp.drop(against_exp.index[against_exp['sample_name'] == truthfilename], inplace=True)
    against_exp['Forward_trim'] = [s.split('R')[0] for s in against_exp['sample_name']]
    against_exp['Forward_trim'] = [s.split('46F')[1] for s in against_exp['Forward_trim']]
    against_exp['Reverse_trim'] = [s.split('R')[1] for s in against_exp['sample_name']]
    against_exp["Forward_trim"] = pd.to_numeric(against_exp["Forward_trim"])
    against_exp["Reverse_trim"] = pd.to_numeric(against_exp["Reverse_trim"])
    against_exp["Forward_trim"].replace({0: 280}, inplace=True)
    against_exp["Reverse_trim"].replace({0: 290}, inplace=True)
    tohm = against_exp.pivot("Forward_trim", "Reverse_trim", "expected_18s_staggered")
    tohm.rename({280: 'full'}, axis=0, inplace=True)
    tohm.rename({290: 'full'}, axis=1, inplace=True)
    ax = sns.heatmap(tohm, cmap=sns.color_palette("hls", 90))
    ax.invert_yaxis()
    plt.figure(figsize=(12,12))
    
    # get max and min values
    print('The min value is {0} and the max value is {1}'.format(df.at[df.stack().index[np.argmin(df.values)]], 
                                                                 minv = df.at[df.stack().index[np.argmax(df.values)]]))

    return (tohm, bacaros_dm)

__r2_plot__ makes a linear regression of the observed relative abundances of each combination of trim lengths against the expected, and plots each coefficient of determination in a heatmap

In [ ]:
## to import the expected taxonomies and transofrm to ratios
def r2_plot(path):
    expected = pd.read_csv(path, sep='\t')
    expected = expected.rename(columns={'silva_taxonomy':'Taxon', 'sample-id': 'Replicate'})
    expected_even = expected[expected.mock_even_insilico != 0]
    expected_even = expected_even.drop(columns=['mock_staggered_insilico','taxonomy'])
    expected_even.reset_index(drop=True, inplace=True)
    expected_even['expected_ratio'] = expected_even['mock_even_insilico']/(expected_even['mock_even_insilico'].sum())
    expected_stagg = expected.drop(columns=['mock_even_insilico','taxonomy'])
    expected_stagg.reset_index(drop=True, inplace=True)
    expected_stagg['expected_ratio'] = expected_stagg['mock_staggered_insilico']/(expected_stagg['mock_staggered_insilico'].sum())
    
    return (expected_even, expected_stagg)

__get_fig_per_group__ makes a heatmap of the observed against expected relative abundances for given group names and colors ratios above the expected in blues, and below the expected in reds.

In [ ]:
def get_fig_per_group(groupname, expectedratio, duplicate='mean'):
    neoceratium = separated[separated['Taxon'].str.contains(groupname)]
    neoceratium = neoceratium[neoceratium.feature_frequency !=0]
    neoceratium.rename(columns = {'sample-id':'sample_id'}, inplace=True)
    if duplicate!='mean':
        neoceratiumR1 = neoceratium[neoceratium.sample_id == 'R46-18S-'+duplicate]
    else:
        neoceratiumR1 = neoceratium.groupby(['Forward_trim','Reverse_trim'])[['ratio']].mean()
        neoceratiumR1 = neoceratiumR1.reset_index()
    neoceratiumR1["Forward_trim"] = pd.to_numeric(neoceratiumR1["Forward_trim"])
    neoceratiumR1["Reverse_trim"] = pd.to_numeric(neoceratiumR1["Reverse_trim"])
    neoceratiumR1["Forward_trim"].replace({0: 280}, inplace=True)
    neoceratiumR1["Reverse_trim"].replace({0: 290}, inplace=True)
    neoceratiumR1merged = neoceratiumR1.groupby(['Forward_trim','Reverse_trim'])[['ratio']].mean()
    neoceratiumR1merged = neoceratiumR1merged.reset_index()
    tohm = neoceratiumR1merged.pivot("Forward_trim", "Reverse_trim", "ratio")
    tohm.rename({280: 'full'}, axis=0, inplace=True)
    tohm.rename({290: 'full'}, axis=1, inplace=True)
    ax = sns.heatmap(tohm, cmap="Oranges_r")#, mask= (tohm < (expectedratio-(0.0005*expectedratio))) & (tohm > (expectedratio+(0.0005*expectedratio)))) #cmap=sns.color_palette("hls", 90)
    ax = sns.heatmap(tohm, mask=tohm <= expectedratio, cmap=sns.color_palette("GnBu", 5)) #square=True, annot=False, vmin=0, vmax=1, cbar=False, ax=ax)
    #ax = sns.heatmap(tohm, mask=tohm >= expectedratio, cmap=sns.color_palette("Oranges_r", 5)) #square=True, annot=False, vmin=0, vmax=1, cbar=False, ax=ax)
    ax.invert_yaxis()
    plt.figure(figsize=(12,12))
    ax.set(xlabel='Reverse trim length', ylabel='Forward trim length')
    fig = ax.get_figure()
    fig.savefig('ratio'+groupname+'.png', bbox_inches = "tight")
    return neoceratiumR1merged

__ttst__ runs one sample t tests between each taxonomic groups observed against expected relative abundances, and plots the results in a boxplot for a single trimming combination

In [ ]:
#run 1 sample t test
#takes trimcombination in format F40R40
def ttst(trimcombination):
    compr_obs_exp = expstagg.merge(separated, how='outer', on='Taxon')
    compr_obs_exp = compr_obs_exp.rename(columns={'sample-id': 'Replicate'})
    compr_obs_exp=compr_obs_exp.fillna(0)
    compr_obs_exp["group"].replace({0: "False positive"}, inplace=True)
    df_with_groups = compr_obs_exp[compr_obs_exp.table_id == trimcombination]
    
    
    means = df_with_groups.groupby('group').mean()
    newsi=means[['expected_ratio', 'ratio']].copy()
    newsi.sort_values('expected_ratio', ascending = False)
    newsi['Difference'] = newsi['expected_ratio'] / newsi['ratio']
    newsi
    
    # generate a boxplot to see the data distribution by treatments. Using boxplot, we can easily detect the differences between different treatments
    ax = sns.boxplot(x='ratio', y='group', data=df_with_groups).set(
    xlabel='Relative abundance', 
    ylabel='Group'
    )
    #ax.tick_params(axis='x', labelrotation=90)
    plt.show()
    
    results = []
    for group in groups:    
        arr = df_with_groups[(df_with_groups['group'] == group)]['ratio'].values  #Filter the dataframe. 
        results.append({'group': group,
                        'ratios': arr}) #Make a single "record" containing the table id, replicate, and ratio array.
        r_gr = pd.DataFrame.from_records(results)
    
    y = expstagg['expected_ratio']
    for i in range(len(y[0:-1])):
        xi = r_gr.iloc[i]['ratios']
        yi = y[i]
        print(r_gr.iloc[i]['group'])
        print(stats.ttest_1samp(xi, yi))
        #but there are outliers
    return (r_gr, newsi)

## Run analyses and make figures

In [ ]:
def get_figures(community={"16S", "18S"}, composition, runnumber, R='all', F='all',level=7, truthfilename):
    df = consolidate_tables(community)
    merged = merge_metada()
    separated_taxonomies, separated_dic = pick_metadata(composition, runnumber, R='all', F='all')
    rename_taxonomies()
    make_fasta()
    make_tbd_hm(level=7)
    r2_plot()
    get_fig_per_group()
    
    
    